# cpDNA Multi-Dataset Preparation for Classical & Quantum ML

This notebook prepares chloroplast DNA windows and encodings for classical & quantum ML.
It supports multiple GenBank accessions, richer provenance, k-mer size control, and a
run journal (Markdown + JSON) for reproducibility.

Pipeline Outline:
1. Performance env + timing utility
2. Setup paths, config & accession collection (env / CSV / default)
3. Fetch & cache GenBank records (retry, validation)
4. Parse features (CDS=1 vs tRNA/rRNA=0) + sequence ambiguity metrics
5. Window generation with ambiguity stats
6. Encodings (one-hot, k-mer, angle) persisted with metadata
7. Stratified splits (pooled + per-dataset optional)
8. Dataset audit tables + global summary
9. Export run journal & provenance manifest

Run order: Execute sequentially after restarting kernel (Cell 0 must run first).


# Cell 0 — perf env (paste first, then restart kernel once)

In [10]:
# Configure BLAS thread counts early to stabilize performance across runs
import os
os.environ.setdefault("OMP_NUM_THREADS", "8")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "8")
os.environ.setdefault("MKL_NUM_THREADS", "8")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "8")
print("BLAS threads:",
      os.environ.get("OMP_NUM_THREADS"),
      os.environ.get("OPENBLAS_NUM_THREADS"),
      os.environ.get("MKL_NUM_THREADS"),
      os.environ.get("NUMEXPR_NUM_THREADS"))

# Lightweight phase timing utility for coarse profiling of notebook stages
import time, json
from contextlib import contextmanager
from collections import defaultdict

class PhaseTimer:
    def __init__(self): self.t = defaultdict(float)
    @contextmanager
    def timed(self, key):
        t0 = time.perf_counter()
        yield
        self.t[key] += time.perf_counter() - t0
    def add(self, key, seconds): self.t[key] += seconds
    def to_dict(self): return dict(self.t)

def pretty_seconds(sec):
    # Present durations in human-friendly units
    return f"{sec/60:.1f} min" if sec >= 60 else f"{sec:.1f} s"

BLAS threads: 8 8 8 8


# Cell 1 — setup & folders (multi-dataset aware)

In [11]:
# Core imports and library setup
from pathlib import Path
import json, os, warnings, sys, time, re
import numpy as np
import pandas as pd
from Bio import Entrez, SeqIO  # NCBI GenBank access & parsing

warnings.filterwarnings("ignore")
np.random.seed(123)

# === Directories ===
ROOT = Path(".")
DATA = ROOT / "data"
RAW = DATA / "raw"
PROCESSED = DATA / "processed"
METADATA = DATA / "metadata"
RESULTS = ROOT / "results"
LOGS = RESULTS / "logs"
for p in [RAW, PROCESSED, METADATA, RESULTS, LOGS]:
    p.mkdir(parents=True, exist_ok=True)

# === Dataset roster (curated Meliaceae cpDNA; Neem + relatives) ===
# You can extend/override via env var NCBI_ACCESSIONS or data/metadata/sources.csv (column 'accession')
CURATED_ACCESSIONS = [
    # Azadirachta (Neem & relatives)
    "KF986530.1",   # Azadirachta indica — primary GenBank
    "NC_023792",    # Azadirachta indica — RefSeq (curated)
    "OK037100",     # Azadirachta excelsa
    "OK037101",     # Azadirachta indica var. siamensis
    "OK037102",     # Azadirachta indica (genus study)

    # Other Meliaceae chloroplast genomes (comparative set)
    "MT460410",     # Melia azedarach
    "MG724915.1",   # Cedrela odorata
    "MG813875",     # Toona ciliata
    "KY923074",     # Entandrophragma cylindricum
    "MF401522",     # Carapa guianensis
    "MH330687",     # Xylocarpus rumphii
    "MK058684",     # Khaya madagascariensis
    "MH348156",     # Swietenia macrophylla
]

# === Runtime configuration ===
default_acc = "KF986530.1"  # Neem cpDNA (anchor)
# env-specified additions (comma-separated)
env_list = [s.strip() for s in os.environ.get("NCBI_ACCESSIONS","").split(",") if s.strip()]

# file-specified additions (data/metadata/sources.csv with column 'accession')
file_list = []
src_csv = METADATA / "sources.csv"
if src_csv.exists():
    try:
        file_list = [str(s).strip() for s in pd.read_csv(src_csv)["accession"].astype(str).tolist() if str(s).strip()]
    except Exception as e:
        print(f"! Could not read {src_csv}: {e} — ignoring file entries.")

def _norm_acc(acc: str) -> str:
    # Keep version suffixes if present (e.g., .1); preserve NC_ prefix.
    return acc.strip()

# Merge with stable order: curated → default → env → file; de-duplicate preserving first occurrence
_seen = set()
ACCESSIONS = []
for source_list in [CURATED_ACCESSIONS, [default_acc], env_list, file_list]:
    for a in source_list:
        a = _norm_acc(a)
        if not a: 
            continue
        if a not in _seen:
            _seen.add(a)
            ACCESSIONS.append(a)

# Optional lightweight accession sanity check (warn only)
_acc_pat = re.compile(r"^(?:[A-Z]{1,2}_?\d{5,}|[A-Z]{2}\d{6,})(?:\.\d+)?$")
for a in ACCESSIONS:
    if not _acc_pat.match(a):
        print(f"⚠️  accession format looks unusual: {a}")

# Email for NCBI Entrez (must be real)
EMAIL = os.environ.get("NCBI_EMAIL", "datdsgunarathna@gmail.com")
Entrez.email = EMAIL

# Sliding window parameters
W = int(os.environ.get("WINDOW_LEN", "256"))
STRIDE = int(os.environ.get("WINDOW_STRIDE", "128"))
KMER_K = int(os.environ.get("KMER_K", "3"))  # k-mer size

print("Folders:", RAW, PROCESSED, RESULTS, sep="\n  - ")
print("Accession list (final, deduped):", ACCESSIONS)
print("NCBI email:", EMAIL, "(OK)" if EMAIL and "@" in EMAIL else "(WARNING: set a real email!)")
print("Window config: W =", W, " STRIDE =", STRIDE, " | k-mer k =", KMER_K)

# === Run journal to document outcomes & issues (saved later as Markdown + JSON) ===
class RunJournal:
    def __init__(self): self.events = []
    def log(self, step, dataset, status, message, **extras):
        self.events.append({
            "ts": time.strftime("%Y-%m-%d %H:%M:%S"),
            "step": step, "dataset": dataset, "status": status, "message": message, **extras
        })
        sym = "✅" if status=="ok" else ("⚠️" if status=="warn" else "❌")
        print(f"{sym} [{step}] {dataset}: {message}")
    def to_frame(self): return pd.DataFrame(self.events)
    def save(self, basepath: Path):
        df = self.to_frame()
        json_path = basepath.with_suffix(".json")
        md_path = basepath.with_suffix(".md")
        df.to_json(json_path, orient="records", indent=2)
        cols = ["ts","step","dataset","status","message"]
        md = ["| " + " | ".join(cols) + " |", "|" + "|".join(["---"]*len(cols)) + "|"]
        for _,r in df[cols].iterrows():
            md.append("| " + " | ".join(str(r[c]) for c in cols) + " |")
        md_path.write_text("\n".join(md), encoding="utf-8")
        print(f"📝 Saved prep journal:\n  - {md_path}\n  - {json_path}")

JOURNAL = RunJournal()

Folders:
  - data\raw
  - data\processed
  - results
Accession list (final, deduped): ['KF986530.1', 'NC_023792', 'OK037100', 'OK037101', 'OK037102', 'MT460410', 'MG724915.1', 'MG813875', 'KY923074', 'MF401522', 'MH330687', 'MK058684', 'MH348156']
NCBI email: datdsgunarathna@gmail.com (OK)
Window config: W = 256  STRIDE = 128  | k-mer k = 3


# Cell 2 — fetch GenBank (cached + retries; multi-dataset)

In [12]:
def fetch_genbank(accession: str, out_file: Path, retries=3, delay=2.0) -> Path:
    """Download GenBank record, cache to disk. Retries on transient failures. [NCBI/GenBank]"""
    if out_file.exists() and out_file.stat().st_size > 0:
        JOURNAL.log("fetch", accession, "ok", f"Using cached {out_file.name}")
        return out_file
    last_err = None
    for k in range(1, retries+1):
        try:
            with Entrez.efetch(db="nuccore", id=accession, rettype="gb", retmode="text") as handle:
                text = handle.read()
            # Basic sanity: GenBank files include LOCUS + ORIGIN sections
            if ("LOCUS" not in text) or ("ORIGIN" not in text):
                raise RuntimeError("Response did not look like a GenBank record (missing LOCUS/ORIGIN).")
            out_file.write_text(text, encoding="utf-8")
            JOURNAL.log("fetch", accession, "ok", f"Downloaded {out_file.name}")
            return out_file
        except Exception as e:
            last_err = e
            JOURNAL.log("fetch", accession, "warn", f"Attempt {k}/{retries} failed: {e}")
            time.sleep(delay * k)  # backoff
    JOURNAL.log("fetch", accession, "fail", f"Giving up after {retries} attempts.")
    raise last_err

gb_paths = {}
for acc in ACCESSIONS:
    p = RAW / f"{acc}.gb"
    try:
        gb_paths[acc] = fetch_genbank(acc, p)
    except Exception as e:
        gb_paths[acc] = None

✅ [fetch] KF986530.1: Using cached KF986530.1.gb
✅ [fetch] NC_023792: Using cached NC_023792.gb
✅ [fetch] OK037100: Using cached OK037100.gb
✅ [fetch] OK037101: Using cached OK037101.gb
✅ [fetch] OK037102: Using cached OK037102.gb
✅ [fetch] MT460410: Using cached MT460410.gb
✅ [fetch] MG724915.1: Using cached MG724915.1.gb
✅ [fetch] MG813875: Using cached MG813875.gb
✅ [fetch] KY923074: Using cached KY923074.gb
✅ [fetch] MF401522: Using cached MF401522.gb
✅ [fetch] MH330687: Using cached MH330687.gb
✅ [fetch] MK058684: Using cached MK058684.gb
✅ [fetch] MH348156: Using cached MH348156.gb


# Cell 3 — parse features (CDS vs RNA) per dataset

In [13]:
# Parse GenBank features; keep CDS=1 and tRNA/rRNA=0; skip pseudogenes.
# Save per-accession features CSV; combine into one DataFrame with dataset metadata.

all_feat_rows = []
meta_rows = []

for acc, gbp in gb_paths.items():
    if gbp is None:
        JOURNAL.log("parse", acc, "fail", "No GenBank file to parse (fetch failed).")
        continue
    try:
        record = SeqIO.read(gbp, "genbank")
        organism = (record.annotations.get("organism") or "").strip() or "unknown"
        seq = record.seq
        seq_len = len(seq)
        # count ambiguous bases
        amb_count = sum(1 for ch in str(seq).upper() if ch not in "ACGT")
        amb_ratio = amb_count / max(1, seq_len)

        rows = []
        for f in record.features:
            if f.type not in ("CDS", "tRNA", "rRNA"):
                continue
            if "pseudo" in f.qualifiers:
                continue
            start, end = int(f.location.start), int(f.location.end)
            lab = 1 if f.type == "CDS" else 0
            rows.append({
                "accession": acc,
                "organism": organism,
                "type": f.type,
                "start": start,
                "end": end,
                "length": end - start,
                "strand": int(f.location.strand or 1),
                "label": lab,
            })
        df_feat = pd.DataFrame(rows).sort_values(["start","end"]).reset_index(drop=True)
        # Save per-dataset features
        out_csv = PROCESSED / f"{acc}_features.csv"
        df_feat.to_csv(out_csv, index=False)

        # Meta & journal
        counts = df_feat["type"].value_counts().to_dict()
        JOURNAL.log("parse", acc, "ok", f"{organism} | len={seq_len:,} | features: {counts} | ambiguous={amb_ratio:.4f}")
        all_feat_rows.append(df_feat)
        meta_rows.append({
            "accession": acc, "organism": organism, "seq_len": seq_len,
            "ambiguous_bases": amb_count, "ambiguous_ratio": round(amb_ratio, 6),
            "n_features": int(len(df_feat)),
            "n_cds": int((df_feat["label"]==1).sum()),
            "n_rna": int((df_feat["label"]==0).sum()),
        })
        if df_feat.empty:
            JOURNAL.log("parse", acc, "warn", "No CDS/tRNA/rRNA features parsed (check annotation quality).")
    except Exception as e:
        JOURNAL.log("parse", acc, "fail", f"Parse error: {e}")

df_feats = pd.concat(all_feat_rows, ignore_index=True) if all_feat_rows else pd.DataFrame(columns=["accession","organism","type","start","end","length","strand","label"])
df_meta = pd.DataFrame(meta_rows)
display(df_meta)

✅ [parse] KF986530.1: Azadirachta indica | len=160,737 | features: {'CDS': 86, 'tRNA': 37, 'rRNA': 8} | ambiguous=0.0000
✅ [parse] NC_023792: Azadirachta indica | len=160,737 | features: {'CDS': 86, 'tRNA': 37, 'rRNA': 8} | ambiguous=0.0000
✅ [parse] OK037100: Azadirachta excelsa | len=160,361 | features: {'CDS': 86, 'tRNA': 37, 'rRNA': 8} | ambiguous=0.0000
✅ [parse] OK037101: Azadirachta indica var. siamensis | len=160,477 | features: {'CDS': 86, 'tRNA': 37, 'rRNA': 6} | ambiguous=0.0000
✅ [parse] OK037102: Azadirachta indica | len=160,876 | features: {'CDS': 85, 'tRNA': 37, 'rRNA': 8} | ambiguous=0.0000
✅ [parse] MT460410: Melia azedarach | len=160,393 | features: {'CDS': 87, 'tRNA': 37, 'rRNA': 8} | ambiguous=0.0000
✅ [parse] MG724915.1: Cedrela odorata | len=158,558 | features: {'CDS': 85, 'tRNA': 37, 'rRNA': 8} | ambiguous=0.0000
✅ [parse] MG813875: Toona ciliata | len=159,502 | features: {'CDS': 89, 'tRNA': 40, 'rRNA': 8} | ambiguous=0.0000
✅ [parse] MG724915.1: Cedrela odorata 

,accession,organism,seq_len,ambiguous_bases,ambiguous_ratio,n_features,n_cds,n_rna
0,KF986530.1,Azadirachta indica,160737,0,0.000000,131,86,45
1,NC_023792,Azadirachta indica,160737,0,0.000000,131,86,45
2,OK037100,Azadirachta excelsa,160361,0,0.000000,131,86,45
3,OK037101,Azadirachta indica var. siamensis,160477,0,0.000000,129,86,43
4,OK037102,Azadirachta indica,160876,0,0.000000,130,85,45
5,MT460410,Melia azedarach,160393,0,0.000000,132,87,45
6,MG724915.1,Cedrela odorata,158558,0,0.000000,130,85,45
7,MG813875,Toona ciliata,159502,0,0.000000,137,89,48
8,KY923074,Entandrophragma cylindricum,159609,0,0.000000,130,85,45
9,MF401522,Carapa guianensis,159483,0,0.000000,131,86,45


# Cell 4 — windows & encodings (multi-dataset; richer logging)

In [14]:
from collections import Counter
import itertools

def one_hot_encode(s: str):
    m = {"A":0, "C":1, "G":2, "T":3}
    X = np.zeros((len(s), 4), dtype=np.float32)
    for i,ch in enumerate(s.upper()):
        j = m.get(ch)
        if j is not None:
            X[i, j] = 1.0
    return X

def kmer_counts(s: str, k=3):
    bases = ["A","C","G","T"]
    kmers = ["".join(p) for p in itertools.product(bases, repeat=k)]
    c = Counter(s[i:i+k] for i in range(len(s)-k+1))
    v = np.array([c.get(km, 0) for km in kmers], dtype=np.float32)
    sm = v.sum()
    return v / sm if sm > 0 else v

def angle_encode(s: str):
    am = {"A":0.0, "C":np.pi/2, "G":np.pi, "T":3*np.pi/2}
    return np.array([am.get(ch.upper(), 0.0) for ch in s], dtype=np.float32)

# Build windows per feature region across datasets
windows, labels, ds_idx = [], [], []  # sequences, labels, dataset-index
dataset_order = sorted(df_feats["accession"].unique().tolist())
acc_to_idx = {a:i for i,a in enumerate(dataset_order)}

per_ds_counts = {a:{"total":0,"pos":0,"neg":0,"short_skipped":0,"ambiguous_chars":0} for a in dataset_order}

# Preload sequences per accession for speed
acc_to_seq = {}
for acc in dataset_order:
    gbp = RAW / f"{acc}.gb"
    try:
        acc_to_seq[acc] = SeqIO.read(gbp, "genbank").seq
    except Exception as e:
        JOURNAL.log("windows", acc, "fail", f"Could not read sequence for windows: {e}")

for r in df_feats.itertuples():
    acc = r.accession
    seq = acc_to_seq.get(acc)
    if seq is None: 
        continue
    sub = str(seq[r.start:r.end])
    if len(sub) < W:
        per_ds_counts[acc]["short_skipped"] += 1
        continue
    for off in range(0, len(sub) - W + 1, STRIDE):
        chunk = sub[off:off+W]
        # track ambiguous characters inside the window
        amb_here = sum(1 for ch in chunk if ch not in "ACGTacgt")
        per_ds_counts[acc]["ambiguous_chars"] += amb_here
        windows.append(chunk)
        labels.append(r.label)
        ds_idx.append(acc_to_idx[acc])
        per_ds_counts[acc]["total"] += 1
        if r.label == 1: per_ds_counts[acc]["pos"] += 1
        else: per_ds_counts[acc]["neg"] += 1

windows = np.array(windows, dtype=object)
y = np.array(labels, dtype=np.int64)
ds_idx = np.array(ds_idx, dtype=np.int32)

# Per-dataset outcome logs
for acc in dataset_order:
    c = per_ds_counts[acc]
    if c["total"] == 0:
        JOURNAL.log("windows", acc, "warn", f"No windows produced. short_skipped={c['short_skipped']}")
    else:
        msg = f"windows={c['total']} | CDS={c['pos']} | RNA={c['neg']} | short_skipped={c['short_skipped']} | ambiguous_chars={c['ambiguous_chars']}"
        JOURNAL.log("windows", acc, "ok", msg)

print("Total windows:", len(windows), "| positive(CDS):", int(y.sum()))

✅ [windows] KF986530.1: windows=1490 | CDS=1375 | RNA=115 | short_skipped=54 | ambiguous_chars=0
✅ [windows] KY923074: windows=1485 | CDS=1370 | RNA=115 | short_skipped=54 | ambiguous_chars=0
✅ [windows] MF401522: windows=1494 | CDS=1377 | RNA=117 | short_skipped=54 | ambiguous_chars=0
✅ [windows] MG724915.1: windows=1483 | CDS=1366 | RNA=117 | short_skipped=54 | ambiguous_chars=0
✅ [windows] MG813875: windows=1486 | CDS=1369 | RNA=117 | short_skipped=61 | ambiguous_chars=0
✅ [windows] MH330687: windows=1496 | CDS=1379 | RNA=117 | short_skipped=54 | ambiguous_chars=6
✅ [windows] MH348156: windows=1453 | CDS=1336 | RNA=117 | short_skipped=53 | ambiguous_chars=0
✅ [windows] MK058684: windows=1491 | CDS=1374 | RNA=117 | short_skipped=57 | ambiguous_chars=4
✅ [windows] MT460410: windows=2746 | CDS=1377 | RNA=1369 | short_skipped=55 | ambiguous_chars=0
✅ [windows] NC_023792: windows=1490 | CDS=1375 | RNA=115 | short_skipped=54 | ambiguous_chars=0
✅ [windows] OK037100: windows=1486 | CDS=136

# Cell 5 — save arrays (combined + metadata)

In [15]:
# Materialize encodings
with PhaseTimer().timed("encode"):
    X_onehot = np.stack([one_hot_encode(w) for w in windows])     # [N, W, 4]
    X_kmer   = np.stack([kmer_counts(w, k=KMER_K) for w in windows])   # [N, 4^k]
    X_angle  = np.stack([angle_encode(w) for w in windows])       # [N, W]

# Save combined artifacts
enc_npz = PROCESSED/"encodings_all.npz"
np.savez_compressed(enc_npz,
                    y=y, onehot=X_onehot, kmer=X_kmer, angle=X_angle,
                    window=W, stride=STRIDE, kmer_k=KMER_K, ds_idx=ds_idx)
pd.DataFrame({"y": y, "ds_idx": ds_idx}).to_csv(PROCESSED/"labels.csv", index=False)
pd.DataFrame({"accession": dataset_order, "ds_idx": list(range(len(dataset_order)))}).to_csv(PROCESSED/"dataset_index.csv", index=False)

meta = {
    "accessions": dataset_order,
    "window": W, "stride": STRIDE, "kmer_k": KMER_K,
    "n_samples": int(len(y)), "n_datasets": int(len(dataset_order)),
}
(Path(PROCESSED/"meta.json")).write_text(json.dumps(meta, indent=2), encoding="utf-8")

JOURNAL.log("save", "all", "ok", f"Saved encodings to {enc_npz.name} with N={len(y)} samples across {len(dataset_order)} datasets.")

✅ [save] all: Saved encodings to encodings_all.npz with N=20560 samples across 13 datasets.


# Cell 6 — splits (pooled + per-dataset)

In [16]:
from sklearn.model_selection import StratifiedShuffleSplit

# --- Pooled stratified splits (default) ---
idx = np.arange(len(y))
tr_idx, te_idx = next(StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42).split(idx, y))
tr_sub, va_idx = next(StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=43).split(tr_idx, y[tr_idx]))
tr_idx = tr_idx[tr_sub]
splits_pooled = {"train": tr_idx.tolist(), "val": va_idx.tolist(), "test": te_idx.tolist()}
(Path(PROCESSED/"splits_pooled.json")).write_text(json.dumps(splits_pooled, indent=2), encoding="utf-8")
JOURNAL.log("split", "pooled", "ok", f"train={len(tr_idx)}, val={len(va_idx)}, test={len(te_idx)}")

# --- Optional: per-dataset stratified splits (useful for within-dataset eval) ---
splits_per_ds = {}
for acc_i, acc in enumerate(dataset_order):
    mask = (ds_idx == acc_i)
    if mask.sum() < 10 or len(np.unique(y[mask])) < 2:
        JOURNAL.log("split", acc, "warn", f"Too few samples or single class (n={int(mask.sum())}); skipping per-dataset splits.")
        continue
    idx_local = np.where(mask)[0]
    y_local = y[mask]
    tr_i, te_i = next(StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=100+acc_i).split(idx_local, y_local))
    tr_i2, va_i = next(StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=200+acc_i).split(idx_local[tr_i], y_local[tr_i]))
    tr_i = idx_local[tr_i[tr_i2]]
    va_i = idx_local[va_i]
    te_i = idx_local[te_i]
    splits_per_ds[acc] = {"train": tr_i.tolist(), "val": va_i.tolist(), "test": te_i.tolist()}
    JOURNAL.log("split", acc, "ok", f"(per-dataset) train={len(tr_i)}, val={len(va_i)}, test={len(te_i)}")

(Path(PROCESSED/"splits_per_dataset.json")).write_text(json.dumps(splits_per_ds, indent=2), encoding="utf-8")
{ "pooled": {k: len(v) for k,v in splits_pooled.items()},
  "per_dataset": {k: {s: len(v[s]) for s in v} for k,v in splits_per_ds.items()} }

✅ [split] pooled: train=12336, val=4112, test=4112
✅ [split] KF986530.1: (per-dataset) train=894, val=298, test=298
✅ [split] KY923074: (per-dataset) train=891, val=297, test=297
✅ [split] MF401522: (per-dataset) train=896, val=299, test=299
✅ [split] MG724915.1: (per-dataset) train=889, val=297, test=297
✅ [split] MG813875: (per-dataset) train=891, val=297, test=298
✅ [split] MH330687: (per-dataset) train=897, val=299, test=300
✅ [split] MH348156: (per-dataset) train=871, val=291, test=291
✅ [split] MK058684: (per-dataset) train=894, val=298, test=299
✅ [split] MT460410: (per-dataset) train=1647, val=549, test=550
✅ [split] NC_023792: (per-dataset) train=894, val=298, test=298
✅ [split] OK037100: (per-dataset) train=891, val=297, test=298
✅ [split] OK037101: (per-dataset) train=887, val=296, test=296
✅ [split] OK037102: (per-dataset) train=888, val=296, test=297


{'pooled': {'train': 12336, 'val': 4112, 'test': 4112},
 'per_dataset': {'KF986530.1': {'train': 894, 'val': 298, 'test': 298},
  'KY923074': {'train': 891, 'val': 297, 'test': 297},
  'MF401522': {'train': 896, 'val': 299, 'test': 299},
  'MG724915.1': {'train': 889, 'val': 297, 'test': 297},
  'MG813875': {'train': 891, 'val': 297, 'test': 298},
  'MH330687': {'train': 897, 'val': 299, 'test': 300},
  'MH348156': {'train': 871, 'val': 291, 'test': 291},
  'MK058684': {'train': 894, 'val': 298, 'test': 299},
  'MT460410': {'train': 1647, 'val': 549, 'test': 550},
  'NC_023792': {'train': 894, 'val': 298, 'test': 298},
  'OK037100': {'train': 891, 'val': 297, 'test': 298},
  'OK037101': {'train': 887, 'val': 296, 'test': 296},
  'OK037102': {'train': 888, 'val': 296, 'test': 297}}}

# Cell 7 — dataset audit (summary + reproducibility checks)

In [17]:
# Build an audit table per dataset and a global row
audit_rows = []
for acc_i, acc in enumerate(dataset_order):
    m = (ds_idx == acc_i)
    n = int(m.sum())
    if n == 0:
        audit_rows.append({
            "accession": acc, "n_windows": 0, "positives": 0, "negatives": 0,
            "pos_ratio": 0.0, "notes": "no_windows"
        })
        continue
    yy = y[m]
    pos = int(yy.sum()); neg = int((yy==0).sum())
    pos_ratio = round(pos / max(1, n), 6)
    notes = []
    if pos == 0 or neg == 0: notes.append("single_class")
    if pos_ratio > 0.9 or pos_ratio < 0.1: notes.append("severe_imbalance")
    audit_rows.append({
        "accession": acc, "n_windows": n, "positives": pos, "negatives": neg,
        "pos_ratio": pos_ratio, "notes": ",".join(notes) if notes else ""
    })

df_audit = pd.DataFrame(audit_rows).sort_values("accession")
df_audit.to_csv(PROCESSED/"dataset_audit.csv", index=False)

# Global row (pooled)
global_audit = {
    "n_samples": int(len(y)),
    "positives": int(y.sum()),
    "negatives": int((y==0).sum()),
    "window": int(W),
    "stride": int(STRIDE),
    "kmer_k": int(KMER_K),
    "n_datasets": int(len(dataset_order)),
}
pd.DataFrame([global_audit]).to_csv(PROCESSED/"dataset_audit_global.csv", index=False)

display(df_audit)
global_audit

,accession,n_windows,positives,negatives,pos_ratio,notes
0,KF986530.1,1490,1375,115,0.922819,severe_imbalance
1,KY923074,1485,1370,115,0.922559,severe_imbalance
2,MF401522,1494,1377,117,0.921687,severe_imbalance
3,MG724915.1,1483,1366,117,0.921106,severe_imbalance
4,MG813875,1486,1369,117,0.921265,severe_imbalance
5,MH330687,1496,1379,117,0.921791,severe_imbalance
6,MH348156,1453,1336,117,0.919477,severe_imbalance
7,MK058684,1491,1374,117,0.921529,severe_imbalance
8,MT460410,2746,1377,1369,0.501457,
9,NC_023792,1490,1375,115,0.922819,severe_imbalance


{'n_samples': 20560,
 'positives': 17793,
 'negatives': 2767,
 'window': 256,
 'stride': 128,
 'kmer_k': 3,
 'n_datasets': 13}

# Cell 8 — export prep journal (what was done, outcomes, problems)

In [18]:
# Save a human-readable log (Markdown + JSON) capturing successes, warnings & failures for documentation
ts = time.strftime("%Y%m%d_%H%M%S")
journal_base = LOGS / f"prep_run_{ts}"
JOURNAL.save(journal_base)

# Also persist a concise run summary for provenance
provenance = {
    "timestamp": ts,
    "accessions": dataset_order,
    "email": EMAIL,
    "window": W, "stride": STRIDE, "kmer_k": KMER_K,
    "files": {
        "encodings": str((PROCESSED/"encodings_all.npz").resolve()),
        "labels": str((PROCESSED/"labels.csv").resolve()),
        "dataset_index": str((PROCESSED/"dataset_index.csv").resolve()),
        "splits_pooled": str((PROCESSED/"splits_pooled.json").resolve()),
        "splits_per_dataset": str((PROCESSED/"splits_per_dataset.json").resolve()),
        "dataset_audit": str((PROCESSED/"dataset_audit.csv").resolve()),
        "dataset_audit_global": str((PROCESSED/"dataset_audit_global.csv").resolve()),
        "journal_md": str((journal_base.with_suffix(".md")).resolve()),
        "journal_json": str((journal_base.with_suffix(".json")).resolve()),
    }
}
(Path(METADATA/f"provenance_{ts}.json")).write_text(json.dumps(provenance, indent=2), encoding="utf-8")
print("Provenance saved:", METADATA/f"provenance_{ts}.json")

📝 Saved prep journal:
  - results\logs\prep_run_20250917_104447.md
  - results\logs\prep_run_20250917_104447.json
Provenance saved: data\metadata\provenance_20250917_104447.json
